In [1]:
import pickle
import pandas as pd

with open('titanic_step4_importance_train.pickle', 'rb') as pickle_filename:
    train_importance = pickle.load(pickle_filename)
with open('titanic_step4_importance_test.pickle', 'rb') as pickle_filename:
    test_importance = pickle.load(pickle_filename)
with open('titanic_step4_importance_train_y.pickle', 'rb') as pickle_filename:
    train_answer = pickle.load(pickle_filename)

In [2]:
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

from sklearn.neighbors import KNeighborsClassifier # 1. KNN
from sklearn.linear_model import LogisticRegression # 2. Logistic
from sklearn.svm import SVC # 3. SVC
from sklearn.tree import DecisionTreeClassifier # 4. Decision Tree
from sklearn.ensemble import RandomForestClassifier # 5. Random Forest
from sklearn.ensemble import ExtraTreesClassifier # 6. Extra Tree
from sklearn.ensemble import GradientBoostingClassifier # 7.GBM
from sklearn.naive_bayes import GaussianNB # 8. GaussianNB
from xgboost import XGBClassifier # 9. Xgboost
from lightgbm import LGBMClassifier # 10. lightgbm
import warnings
warnings.filterwarnings('ignore')

### 하이퍼 파라미터 튜닝과 SVC
- SVC 는 결국 분류의 경계가 되는 경계선을 작성하여, 분류를 실행하는 모델
- 해당 경계선을 일직선으로 할지, 어느 정도 곡률을 가진 선으로 할지도 선정 가능
- 주요 하이퍼 파라미터
  - C : regularization 파라미터
  - gamma: 어느 정도 훈련 셋에 fit 하게 할지를 결정

### RandomizedSearchCV + SVC
- RandomizedSearchCV() 는 랜덤하게 파라미터값을 선정하여, 테스트를 수행하므로,
- 보다 적합한 파라미터값을 도출하기 위해서는 수행횟수를 늘려야 함
- 따라서 머신러닝 모델의 수행성능에 따라, RandomizedSearchCV() 사용시, 수행시간이 상당히 오래 걸릴 수 있으므로,
- RandomizedSearchCV() 이해를 위해서만 일부 모델에서만 테스트를 진행하고,
- GridSearchCV() 을 주로 사용하기로 함

### 참고: 균일 분포 또는 균등 분포(Uniform Distribution)
  - 정해진 범위에서 모든 확률이 균일한 분포를 의미함
  - 균일 분포는 이산형 확률 분포와 연속형 확률 분포 두 형태가 존재
  - 연속형 확률 분포: 두 점 a,b 사이의 연속적인 값에 대한 확률 분포
  - 이산형 확률 분포: 두 점 a,b 사이에 갯수가 정해진 값들에 대한 확률 분포

### stats.uniform(loc, scale)
- loc 부터, loc + scale 까지의 범위에서 균등한 확률로 연속형 값을 추출
- 해당 객체는 rvs() 메서드를 가지고 있고, 이를 사용해서, RandomizedSearchCV() 가 랜덤 값을 균등 확률로 추출해서, 적용 및 테스트

In [3]:
hyperparams = {
    "C": stats.uniform(0, 50),
    "gamma": stats.uniform(0, 1)
}
gd = RandomizedSearchCV(
    estimator=SVC(random_state=1),
    param_distributions=hyperparams,
    n_iter=100,
    cv=5, # 내부적으로 Stratified K Fold 사용
    scoring='accuracy',
    random_state=1,
    n_jobs=-1
)

gd.fit(train_importance, train_answer.values.ravel())
print(gd.best_score_)
print(gd.best_params_)

df = pd.DataFrame(gd.cv_results_)
print(df[['params', 'mean_test_score']])

0.8372795179210344
{'C': 2.995884475610583, 'gamma': 0.12134345574073735}
                                               params  mean_test_score
0   {'C': 20.8511002351287, 'gamma': 0.72032449344...         0.820451
1   {'C': 0.005718740867244332, 'gamma': 0.3023325...         0.616163
2   {'C': 7.337794540855652, 'gamma': 0.0923385947...         0.836162
3   {'C': 9.313010568883545, 'gamma': 0.3455607270...         0.821549
4   {'C': 19.838373711533496, 'gamma': 0.538816734...         0.822685
..                                                ...              ...
95  {'C': 13.164838524355549, 'gamma': 0.065961090...         0.836168
96  {'C': 36.753298164433474, 'gamma': 0.772178029...         0.814839
97  {'C': 45.3907926251762, 'gamma': 0.93197206919...         0.805875
98  {'C': 0.6975786487798508, 'gamma': 0.234362086...         0.831643
99  {'C': 30.83891785008288, 'gamma': 0.9490163206...         0.805875

[100 rows x 2 columns]


### SVC + 하이퍼 파라미터 튜닝 (GridSearchCV 사용)
> 하이퍼파라미터는 일반적으로 적절한 범위가 없기 때문에, 각 데이터에 맞춰서 성능이 나오는 범위를 감으로 지정해야 함
> RandomizedSearchCV() 를 통해, 대략적인 범위를 알아낸 후, 이를 기반으로 GridSearchCV() 를 사용하여
> 범위와, 최적의 값을 가질 수 있는 후보군을 지정하는 방식으로, 최적의 하이퍼파라미터 값을 찾아가는 방법을 많이 사용함

In [4]:
# 파라미터 그리드 셋팅
hyperparams = {
    'C': [10, 15, 20, 23, 25, 30, 50],
    'gamma': [0.001, 0.01, 0.05, 0.06, 0.07, 0.08]
}

# 교차 검증
gd = GridSearchCV(
    estimator=SVC(random_state=1),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

gd.fit(train_importance, train_answer.values.ravel())
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 42 candidates, totalling 210 fits
0.8372920720607621
{'C': 15, 'gamma': 0.06}


### 5. Gradient Boosting Classifier 주요 하이퍼 파라미터
- learning_rate는 학습률을 의미하며, 각 트리의 오류에 기반해서, 어느 정도 수정할지의 비율을 의미
- n_estimator 는 트리의 갯수를 의미
- max_depth 는 트리의 깊이를 의미

### Gradient Boosting Classifier + 하이퍼 파라미터 튜닝 (GridSearchCV 사용)
- Gradient Boosting Classifier 는 수행시간이 오래 걸리므로, RandomizedSearchCV() 로 사전 테스트를 통해, 대략적인 최적의 파라미터값을 예상하여, GridSearchCV() 로 테스트

In [5]:
learning_rate=[0.01, 0.05, 0.1, 0.2]
n_estimators=[100, 1000, 2000]
max_depth=[3, 5, 10, 15]

hyperparams = {
    'learning_rate': learning_rate,
    'n_estimators': n_estimators,
    'max_depth': max_depth
}

gd = GridSearchCV(
    estimator=GradientBoostingClassifier(random_state=1),
    param_grid=hyperparams,
    verbose=True,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

gd.fit(train_importance, train_answer.values.ravel())
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
0.8406377502981608
{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 1000}


### 6. Logistic Regression 주요 하이퍼 파라미터
* penalty: regularization 종류 선정 (l1, l2 등)
* C: regularization 적용 강도

### Logistic Regression + 하이퍼 파라미터 튜닝 (RandomizedSearchCV 사용)

In [6]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

hyperparams = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': stats.uniform(0, 1000)
}
gd = RandomizedSearchCV(
    estimator=LogisticRegression(random_state=1, solver='lbfgs', max_iter=1000),
    param_distributions=hyperparams,
    n_iter=100,
    cv=5,
    scoring='accuracy',
    random_state=1,
    n_jobs=-1
)
gd.fit(train_importance, train_answer.values.ravel())
print(gd.best_score_)
print(gd.best_params_)

0.8384282217061075
{'C': 18.288277344191805, 'penalty': 'l2'}


### Logistic Regression + 하이퍼 파라미터 튜닝 (GridSearchCV 사용)
- numpy.linspace(start, end, num)
  - start ~ end 사이의 값을 등간격으로 num 갯수만큼의 배열을 생성하는 numpy 메서드

In [7]:
np.linspace(700, 900, 10)

penalty = ['l1', 'l2']
C = np.linspace(700, 900, 200)

hyperparams = {
    'penalty': penalty,
    'C': C
}

gd=GridSearchCV(
    # 2022.12.22 해당 라이브러리 변경으로 인터페이스가 변경되어,  기존 코드로 경고표시가 나옵니다.
    #  경고표시가 나오더라도, 진행에는 문제가 없지만, 당황하실 수 있어서, 경고표시가 나오지 않도록 코드를 업데이트합니다.
    # estimator = LogisticRegression(random_state=1)
    estimator = LogisticRegression(random_state=1, solver='lbfgs', max_iter=1000),
    param_grid = hyperparams,
    verbose=True,
    cv=5,
    scoring = "accuracy",
    n_jobs=-1
)
# 2022.12.22 해당 라이브러리 변경으로 인터페이스가 변경되어,  기존 코드로 경고표시가 나옵니다.
#  경고표시가 나오더라도, 진행에는 문제가 없지만, 당황하실 수 있어서, 경고표시가 나오지 않도록 코드를 업데이트합니다.
# gd.fit(train_importance, train_answer)
gd.fit(train_importance, train_answer.values.ravel())
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits
0.8372983491306257
{'C': 700.0, 'penalty': 'l2'}


### 7. XGBoost 와 주요 하이퍼 파라미터
- 일반적인 XGBoost 하이퍼 파라미터 튜닝 전략
   - ensemble 방식은 수행시, 각 파라미터를 적절히 맞춰주기 때문에, 하이퍼 파라미터 튜닝이 정확도를 높이는데 있어서, 큰 기여를 하는 편은 아니며,
   - 무수히 많은 파라미터가 있고, 수행 시간이 오래 걸리므로, 주요한 파라미터들만 중심으로 튜닝을 진행하는 편이 좋음
- 성능에 영향을 많이 끼치는 주요 파라미터
   - learning_rate
      - 이전 결과를 얼마나 반영할지에 대한 학습 단계별 적용할 가중치를 의미함
      - 일반적으로 0.01 ~ 0.2 사이의 값을 많이 사용함
   - max_depth
      - 트리의 최대 깊이를 의미함
      - 트리의 최대 깊이로 -1 로 하면, 깊이에 제한을 두지 않음
      - 일반적으로 3 ~ 10 사이의 값을 많이 사용함
   - gamma
      - 일종의 정규화(regularization) 파라미터로, gamma 가 높을 수록, regularization 이 높다고 이해하면 됨
      - 트리에서 가지를 추가로 만들기 위해 필요한 최소 loss 기준값으로, gamma 값이 작으면, 트리에 보다 많은 가지가 만들어진다고 이해하면 됨
      - 일반적으로 0 이상의 값을 가짐
   - min_child_weight
      - 트리에서 가지를 추가로 만들어 분할하기 위해, 필요한 최소한의 샘플 수
      - 값이 적을 수록, 트리가 더 분할될 수 있음
      - 일반적으로 0 이상의 값을 가짐
   - subsample
      - 각 트리마다 모든 훈련데이터를 사용해서 트리를 만들지 않음
      - 훈련 데이터의 일부를 사용해서 트리를 만든다면, 보다 많은 트리를 만들 수 있고, 이를 통해 트리의 다양성을 높일 수 있음
      - subsample 은 이 때, 각 트리마다 어느 정도의 훈련 데이터 비율을 사용해서 트리를 만들지를 그 비율을 정하는 것임
      - 일반적으로 0.5 ~ 1 사이의 값을 많이 사용함
   - colsample_bytree
      - subsample 과 마찬가지로, 훈련 데이터에서 일부 feature (컬럼) 들만 뽑아서 트리를 만드는 방식
      - 일부 feature (컬럼)들만 뽑아서 트리를 만든다면, 보다 많은 트리를 만들 수 있고, 이를 통해 트리의 다양성을 높일 수 있음
      - colsample_bytree 는 이를 위해 각 트리를 만들 때 사용할 feature의 비율을 정하는 것임
      - 일반적으로 0.5 ~ 1 사이의 값을 많이 사용함

   - reg_alpha: L1 정규화(regularization) 가중치
   - reg_lambda: L2 정규화(regularization) 가중치

In [8]:
!pip install bayesian-optimization==1.4.0

### Bayesian Optimization XGBoost 적용

In [9]:
from bayes_opt import BayesianOptimization
pbounds = {
    'learning_rate': (0.01, 0.5),
    'n_estimators': (100, 1000),
    'max_depth': (3, 10),
    'min_child_weight': (0, 10),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'gamma': (0, 5)
    # 'reg_lambda': (0, 1000, 'log-uniform'),
    # 'reg_alpha': (0, 1.0, 'log-uniform')
}

def xgboost_hyper_param(learning_rate, n_estimators, max_depth, min_child_weight, subsample, colsample_bytree, gamma):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    clf = XGBClassifier(
        max_depth=max_depth,
        min_child_weight= min_child_weight,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        random_state=1,
        eval_metric='logloss'
        # reg_alpha=reg_alpha,
        # reg_lambda=reg_lambda
    )
    return np.mean(cross_val_score(clf, train_importance, train_answer, cv=5, scoring='accuracy'))

optimizer = BayesianOptimization( f=xgboost_hyper_param, pbounds=pbounds, random_state=1)
# init_points: 초기 랜덤 포인트 갯수
# acq='ei': Expected Improvement
# xi=0.01: exploration(불확실성이 가장 높은 점 근처에 최적값이 존재할 것이라는 가정으로 계산된 값) 강도 (보통 0.01)
optimizer.maximize(init_points=10, n_iter=100, acq='ei', xi=0.01)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.8204    | 0.7085    | 3.602     | 0.01006   | 5.116     | 1.468     | 183.1     | 0.5931    |
| 2         | 0.8283    | 0.6728    | 1.984     | 0.274     | 5.934     | 6.852     | 284.0     | 0.9391    |
| 3         | 0.8305    | 0.5137    | 3.352     | 0.2145    | 6.911     | 1.404     | 278.3     | 0.9004    |
| 4         | 0.7969    | 0.9841    | 1.567     | 0.3492    | 9.135     | 8.946     | 176.5     | 0.5195    |
| 5         | 0.8126    | 0.5849    | 4.391     | 0.05819   | 5.948     | 9.579     | 579.8     | 0.8459    |
| 6         | 0.8272    | 0.6578    | 3.433     | 0.419     | 3.128     | 7.501     | 990.0     | 0.8741    |
| 7         | 0.8081    | 0.6402    | 3.946     | 0.06058   | 6.135     | 9.086     | 364.3     | 0.6439    |
| 8       

### Bayesian Optimization LightGBM 적용

In [10]:
import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

pbounds = {
    'learning_rate': (0.01, 0.5),
    'n_estimators': (100, 1000),
    'max_depth': (3, 10),
    'min_child_weight': (0, 10),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0)
    # 'reg_lambda': (0, 1000),
    # 'reg_alpha': (0, 1.0)
}

def lgbm_hyper_param(learning_rate, n_estimators, max_depth, min_child_weight, subsample, colsample_bytree):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    clf = LGBMClassifier(
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=1
        # reg_lambda=reg_lambda,
        # reg_alpha=reg_alpha
    )
    return np.mean(cross_val_score(clf, train_importance, train_answer, cv=5, scoring='accuracy'))   # cv 도 숫자로 작성하여, 내부적으로 (Stratified)KFold 사용함

optimizer = BayesianOptimization( f=lgbm_hyper_param, pbounds=pbounds, verbose=1, random_state=1)
optimizer.maximize(init_points=10, n_iter=100, acq='ei', xi=0.01)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 2         | 0.8227    | 0.5931    | 0.1793    | 5.777     | 5.388     | 477.3     | 0.8426    |
| 4         | 0.8249    | 0.5702    | 0.1071    | 8.605     | 9.683     | 382.1     | 0.8462    |
| 9         | 0.8339    | 0.6439    | 0.07371   | 3.136     | 6.788     | 290.5     | 0.6328    |


### 9. Grid Search XGBoost 적용

### Grid Search XGBoost 적용 1단계
- 주요 파라미터를 모두 넣을 경우, 수행시간이 매우 길어지므로, 3단계로 나누어서 테스트

In [11]:
learning_rate = [0.001, 0.005, 0.01, 0.05, 0.06, 0.1, 0.12, 0.15, 0.17, 0.2]
n_estimators = [10, 50, 60, 75, 85, 100, 125, 150, 200, 250, 500, 1000]

hyperparams = {
    'learning_rate': learning_rate,
    'n_estimators': n_estimators
}

gd=GridSearchCV(
    estimator = XGBClassifier(random_state=1, eval_metric='logloss'),
    param_grid = hyperparams,
    verbose=True,
    cv=5,
    scoring = "accuracy",
    n_jobs=-1
)

gd.fit(train_importance, train_answer)
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
0.8439897056054233
{'learning_rate': 0.005, 'n_estimators': 250}


Grid Search XGBoost 적용 2단계

In [12]:
max_depth = [3, 4, 5, 6, 7, 8, 9, 10]
min_child_weight = [1, 2, 3, 4, 5, 6, 7]

hyperparams = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight
}

gd=GridSearchCV(
    estimator = XGBClassifier(learning_rate=0.17, n_estimators=10, random_state=1, eval_metric='logloss'),
    param_grid = hyperparams,
    verbose=True,
    cv=5,
    scoring = "accuracy",
    n_jobs=-1
)

gd.fit(train_importance, train_answer)
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 56 candidates, totalling 280 fits
0.8439897056054233
{'max_depth': 6, 'min_child_weight': 1}


Grid Search XGBoost 적용 3단계

In [13]:
gamma =  [i*0.1 for i in range(0,5)]
subsample = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
colsample_bytree = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
reg_alpha = [1e-5, 1e-2, 0.1, 1, 100]

hyperparams = {
    'gamma': gamma,
    'subsample':subsample,
    'colsample_bytree':colsample_bytree,
    'reg_alpha': reg_alpha
}

gd=GridSearchCV(
    estimator = XGBClassifier(
        learning_rate=0.17,
        n_estimators=10,
        max_depth=6,
        min_child_weight=1,
        random_state=1,
        eval_metric='logloss'
    ),
    param_grid = hyperparams,
    verbose=True,
    cv=5,
    scoring = "accuracy",
    n_jobs=-1
)

gd.fit(train_importance, train_answer)
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 2025 candidates, totalling 10125 fits
0.8462306195467957
{'colsample_bytree': 0.9, 'gamma': 0.0, 'reg_alpha': 0.1, 'subsample': 0.85}


### KNN

In [14]:
n_neighbors = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
hyperparams = {
    'n_neighbors': n_neighbors
}

gd=GridSearchCV(
    estimator = KNeighborsClassifier(),
    param_grid = hyperparams,
    verbose=True,
    cv=5,
    scoring = "accuracy",
    n_jobs=-1
)

gd.fit(train_importance, train_answer)
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
0.8283095850856819
{'n_neighbors': 6}


### Random Forest.

In [15]:
n_estimators = [10, 50, 100, 200]
max_depth = [3, None]
max_features = [0.1, 0.2, 0.5, 0.8, 'sqrt', 'log2'] # feature 수
min_samples_split = [2, 4, 6, 8, 10] # 노드를 분할하기 위한 최소 샘플 수
min_samples_leaf = [2, 4, 6, 8, 10] # 리프 노드가 되기 위해 필요한 최소 샘플 수

hyperparams = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
}

gd=GridSearchCV(
    estimator = RandomForestClassifier(random_state=1),
    param_grid = hyperparams,
    verbose=True,
    cv=5,
    scoring = "accuracy",
    n_jobs=-1
)

gd.fit(train_importance, train_answer)
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits
0.8484715334881676
{'max_depth': None, 'max_features': 0.8, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}


### Extra Trees

In [16]:
n_estimators = [10, 50, 100, 200]
max_depth = [3, None]
max_features = [0.1, 0.2, 0.5, 0.8, 'sqrt', 'log2'] # feature 수
min_samples_split = [2, 4, 6, 8, 10] # 노드를 분할하기 위한 최소 샘플 수
min_samples_leaf = [2, 4, 6, 8, 10] # 리프 노드가 되기 위해 필요한 최소 샘플 수

hyperparams = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
}

gd=GridSearchCV(
    estimator = ExtraTreesClassifier(random_state=1),
    param_grid = hyperparams,
    verbose=True,
    cv=5,
    scoring = "accuracy",
    n_jobs=-1
)

gd.fit(train_importance, train_answer)
print(gd.best_score_)
print(gd.best_params_)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits
0.8529721925805035
{'max_depth': None, 'max_features': 0.2, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
